# SVMs from Scratch #
    - Matt Robinson

I have an opinion that is perhaps better left unsaid, but I'll say it anyway: most machine learning books are absolutely terrible at explaining SVMs (Support Vector Machines). Most books mention things like the maximum margin and Lagrange multipliers, but fail to adequetly explain many of the steps in the math. To be fair, many of the books are not purporting to be lengthy math treatises. However, I do think the steps should be clear and there should be more intuition for choosing the "widest street" besides "that one looks the best" or "that's the line you would probably draw."

The problem with saying all of these things is that now I am going to try to explain SVMs, and I'll probably do a poor job of it. My tip to all readers is to go read *Pattern Recognition and Machine Learning* by Bishop (2006) or *Learning from Data* by Abu-Mostafa *et al.* (2012). These books are excellent and do by far the best job of explaining the concepts.

Since these books are amazing, I am actually just going to cover the material that I found extra-confusing about *Hard-margin* SVMs. I repeat, this tutorial will not be extensive, but will instead try to clear up the tricky points. If you are confused, much of the material is derived from the aforementioned books -- I again suggest you give them a read.

## Why is a Fatter Margin Better? ###

*Learning from Data* (2012) includes a great, lengthy discussion on this topic. However, I will include just one figure detailing what I find the most intuitively pleasing reason: 

Note the following figures are taken from the slides from the Learning from Data [course website](https://work.caltech.edu/lectures.html#lectures). The book has better figures, but it has frequent warnings not to distribute the material anywhere...
<img src="img/learning_from_data_svm_no_noise.png" width="800" height="600">


Now most of us would say the linear discriminant on the right is the most reasonable choice. But why? Well lets imagin that each data point has some associated noise or measurement error. Let's now plot how much measurement noise/error in the samples each margin allows (noise/error is represented by the black circles): 

<img src="img/learning_from_data_svm_noise.png" width="800" height="600">

The right-most margin (the maximum margin) clearly is more robust to error or noise. Even a little bit of measurement error could result in a data point being incorrectly classified in the left most plot. However, a point with slight measurement error will still likely be classified by the classifier in the right plot. 

All of this can be made more formal, but we will settle for this justification for now. 

# The Math #

Okay, time to get into the thick of it -- the part where most of the testbooks lost me. There are a lot of little small tricks and assumptions behind the math of SVMs; I'll try to make most of the explicit. 

At it's core, a hard-margin SVM is really just a simple linear discriminant. We have $N$ $D$-dimensional input vectors $\mathbf{x}_1,...,\mathbf{x}_N \in \mathbb{R}^D$. We use a linear function $y(\mathbf{x})$ to transform the vectors so that they can then be classified.

$$
y(\mathbf{x}) = \mathbf{w}^T\mathbf{x} + b
$$

where $\mathbf{w}$ is the weights vector and $b$ is the bias.

All points for which $y(\mathbf{x}) \geq 0$ are classified as $+1$ and all points $y(\mathbf{x}) \leq 0$ are classified as $-1$. The decision boundary, therefore, consists of all those points for which $y(\mathbf{x}) = \mathbf{w}^T\mathbf{x} + b = 0$. Let's now dig into some of the math questions that stumped me.

#### Why is $\mathbf{w}$ orthogonal to the decision boundary? ####

Let's start by considering the following figure depicting a linear discriminant function in 2-dimensions from Bishop.

<img src="img/Bishop_2D_discriminant.png" width="600" height="600">

The decsion surface is shown in red, and note that Bishop uses $w_o$ to denote the bias parameter, which we are calling $b$.

Now if we pick two generic inputs $\mathbf{x}_1$ and $\mathbf{x}_2$ that lie on the decision surface, we automatically have that $y(\mathbf{x}_1)=y(\mathbf{x}_2)=0$ since they are on the surface. Furthermore, the vector going from $\mathbf{x}_1$ to $\mathbf{x}_2$ is in the direction of $(\mathbf{x}_2 - \mathbf{x}_1)$ and is parallel to the decision surface. But we also know that, 

$$
y(\mathbf{x}_1)=\mathbf{w}^T\mathbf{x}_1 + b =0=\mathbf{w}^T\mathbf{x}_2 + b = y(\mathbf{x}_2) \\ \implies \mathbf{w}^T(\mathbf{x}_2-\mathbf{x}_1)=0
$$

which shows that $\mathbf{x}$ is orthogonal to every vector in the lying in the decision surface. 

#### Why does $\frac{1}{||\mathbf{w}||}$  represent the width of the margin? ####

Consider again the figure above. The generic vector $\mathbf{x}$ can be decomposed as follows:

$$
\mathbf { x } = \mathbf { x } _ { \perp } + r \frac { \mathbf { w } } { \| \mathbf { w } \| }
$$

where $\mathbf { x } _ { \perp }$ is the orthogonal projection of $\mathbf{x}$ on the decision surface and $r$ is a scalar giving the perpendicular distance from the deicsion surface to $\mathbf{x}$. Note that $r$ is multiplied by the unit vector $\frac { \mathbf { w } } { \| \mathbf { w } \| }$, which is necessarily orthogonal to the decision surface, as shown above.

Let's work with this equation a bit:

$$
\mathbf { x } = \mathbf { x } _ { \perp } + r \frac { \mathbf { w } } { \| \mathbf { w } \| } \\
\begin{align*}
&\implies \mathbf{w}^T\mathbf { x } = \mathbf{w}^T\mathbf { x } _ { \perp } + r { \| \mathbf { w } \| } \\
&\implies \mathbf{w}^T\mathbf { x } + b = \mathbf{w}^T\mathbf { x } _ { \perp } + b + r { \| \mathbf { w } \| } \\
&\implies y(\mathbf{x}) = y(\mathbf { x } _ { \perp }) + r { \| \mathbf { w } \| } \\
&\implies y(\mathbf{x}) = 0 + r { \| \mathbf { w } \| } \\
&\implies r = \frac{y(\mathbf { x })}{ \| \mathbf { w } \| } 
\end{align*}
$$

where we used the fact that $y(\mathbf { x } _ { \perp })=0$ because $\mathbf { x } _ { \perp }$ is on the decision surface.

This value $r$ will be positive above the decision boundary and negative below it, as dictated by the sign of $y(\mathbf { x })$. The trick now is to choose a scaling of $\mathbf{w}$ and $b$ such that $y(\mathbf { x })=1$ for the closest points to the boundary. This will insure that our margin can simply be represented as $\frac{1}{||\mathbf{w}||}$. More on this in the next section.

#### To the actual math. Lots of Lagrange Multipliers ####

To properly explain the SVM, we are going to again start with a simple linear classifier:

$$
y ( \mathbf { x } ) = \mathbf { w } ^{ T } ( \mathbf { x } ) + b
$$

As before, we have $N$ input vectors $\mathbf{x}_1,...,\mathbf{x}_N$ and $N$ corresponding target values $t_1,...,t_N$, where each $t _ { i } \in \{ - 1,1 \}$. For simplicitiy, we will assume **linear separability**. That is, we assume there are choices for the parameters $\mathbf{w}$ and $b$ that will give $t _ { i } y \left( \mathbf { x } _ { i } \right) > 0$ for all data points in the training set. In other words, all points for which $t_i=+1$ will be correctly classified by $y \left( \mathbf { x } _ { i } \right) > 0$, and all points with $t_i=-1$ will satisfy $y \left( \mathbf { x } _ { i } \right) < 0$.

As the linear perceptron showed, there are likely many choices of $\mathbf{w}$ and $b$ that produce a separating decision coundary of the linear separable training data. However, here we are hoping to find the decision boundary that *maximizes the margin*.

Earlier we showed that the distance between the separating hyperplane and a data point $\mathbf{x}$ is given by $\frac{|y(\mathbf { x })|}{ \| \mathbf { w } \| }$. Furthermore, we assumed that all points will be correctly classified with $t _ { i } y \left( \mathbf { x } _ { i } \right) > 0$. Since all the $t_i$ have the property $|t_i|=1$, we arrive at the following:

$$
\frac{|y(\mathbf { x }_i)|}{ \| \mathbf { w } \| } = \frac { t _ { i } y \left( \mathbf { x } _ { i } \right) } { \| \mathbf { w } \| } = \frac { t _ { i } \left( \mathbf { w } ^ { \mathrm { T } }   \mathbf { x } _ { i }  + b \right) } { \| \mathbf { w } \| }
$$

We define the margin as the perpendicular distance to $\mathbf{x}_n$, the closes point in the training set to the separating hyperplane. Therefore, we wish to *maximize* this *minimum distance* to the hyperplane. That optimization problem looks as follows:

$$
\underset { \mathbf { w } , b } { \arg \max } \left\{  \min _ { n }\frac { 1 } { \| \mathbf { w } \| } \left[ t _ { n } \left( \mathbf { w } ^ { \mathrm { T } } \boldsymbol { \phi } \left( \mathbf { x } _ { n } \right) + b \right) \right] \right\}
$$

Note that we can pull the factor $\frac { 1 } { \| \mathbf { w } \| }$ outside the minimzation over $n$ since $\| \mathbf { w } \|$ does not depend on $n$.

It turns out that the above problem is quite hard; therfore, we construct an equivalent problem that is much easier. Our first simplification is to note that we can free rescale the parameters $\mathbf { w } \rightarrow \kappa \mathbf { w } \text { and } b \rightarrow \kappa b$, without chagning the distance from $\mathbf{x_n}$ to the separating hyperplane, $t _ { n } y \left( \mathbf { x } _ { n } \right) / \| \mathbf { w } \|$. (This is obvious if you do it out and realize that you get a factor of $\kappa$ in both the numerator and denominator.)

Using this rescaling, we can arrive at the following simple, *canonical representation*:

$$
t _ { n } \left( \mathbf { w } ^ { \mathrm { T } } \mathbf { x } _ { n } + b \right) = 1 \\
t _ { n } \left( \mathbf { w } ^ { \mathrm { T } } \mathbf { x } _ { i } + b \right) \geqslant 1 , \quad  i \neq n
$$

For those data points that satisfy the equality (first line above), the constraint is said to be *active*. After maximizing the margin, we expect there to be $2$ active constraints for the closest data points. For the rest of the points satisfying the inequlaity (second line above), the constraint is said to be *inactive*.

Another way we make this problem simpler is by turning the maximization of $\frac { 1 } { \| \mathbf { w } \| }$ into the equivalent problem of *miniziming* the quantity $\frac { 1 } { 2 } \| \mathbf { w } \| ^ { 2 }$. Thus we seek 

$$
\underset { \mathbf { w } , b } { \arg \min } \frac { 1 } { 2 } \| \mathbf { w } \| ^ { 2 }
$$

subject to the active and inactive linear constraints we detailed above. The problem thus becomes a Lagrange multiplier problem with $N$ Lagrange multipliers $\lambda_i \geq 0$. The Lagrangian function thus looks as follows:

$$
L ( \mathbf { w } , b , \mathbf {\lambda } ) = \frac { 1 } { 2 } \| \mathbf { w } \| ^ { 2 } - \sum _ { i = 1 } ^ { N } \lambda _ { i } \left\{ t _ { i } \left( \mathbf { w } ^ { \mathrm { T } }  \mathbf { x } _ { i } + b \right) - 1 \right\}
$$

If you have never dealt with Lagrange multipliers involving inequalities before, a little explanation is perhaps in order. For a detailed explanation, I hightly suggest Appendix E of Bishop's book; it is excellent. The short of it is that constrained optimization problems with inequality constraints like above satisfy the KKT (Karush-Kuhn-Tucker) conditions. These conditions require for all data points $\mathbf{x}_i$:

$$
\begin{aligned} \lambda _ { i } & \geqslant 0 \\ t _ { i } y \left( \mathbf { x } _ { i } \right) - 1 & \geqslant 0 \\ \lambda _ { i } \left\{ t _ { i } y \left( \mathbf { x } _ { i } \right) - 1 \right\} & = 0 \end{aligned}
$$

These condition come about because our constraint is now a region, and not just a curve. From the last KKT condition, we see that each point must either satisfy the active contraint $ t _ { i } y \left( \mathbf { x } _ { i } \right)=1$, or have $\lambda_i=0$. Thus, those points that are *inactively* constrained will have $\lambda_i=0$, and will not enter into the sum of our constrained optimization problem. This is why we end up only having to care about the *support vectors* that lie on the margin.

Now back to our constrained optimization problem below:

$$
L ( \mathbf { w } , b , \mathbf {\lambda } ) = \frac { 1 } { 2 } \| \mathbf { w } \| ^ { 2 } - \sum _ { i = 1 } ^ { N } \lambda _ { i } \left\{ t _ { i } \left( \mathbf { w } ^ { \mathrm { T } }  \mathbf { x } _ { i } + b \right) - 1 \right\}
$$

We proceed by taking derivatives with respect to the parameters $\mathbf{w}$ and $b$, then setting the derivatives equal to $0$.

$$
\frac{d L ( \mathbf { w } , b , \mathbf {\lambda } )}{d \mathbf{w}} =\mathbf { w }  - \sum _ { i = 1 } ^ { N } \lambda _ { i }  t _ { i } \mathbf { x } _ { i } \stackrel{\text{set}}{=} 0 \\
\implies \mathbf { w } = \sum _ { i = 1 } ^ { N } \lambda _ { i }  t _ { i } \mathbf { x } _ { i }
$$

$$
\frac{d L ( \mathbf { w } , b , \mathbf {\lambda } )}{d \mathbf{b}} = - \sum _ { i = 1 } ^ { N } \lambda _ { i }  t _ { i } \stackrel{\text{set}}{=} 0 \\
\implies \sum _ { i = 1 } ^ { N } \lambda _ { i }  t _ { i } = 0
$$

We now plug these quantities into $L ( \mathbf { w } , b , \mathbf {\lambda } )$:

$$
L ( \mathbf { w } , b , \mathbf {\lambda } ) = \frac { 1 } { 2 } \| \mathbf { w } \| ^ { 2 } - \sum _ { i = 1 } ^ { N } \lambda _ { i } t _ { i } \mathbf { w } ^ { \mathrm { T } }  \mathbf { x } _ { i } + \lambda _ { i } t _ { i } b  - \lambda_i
$$

$$
= \frac { 1 } { 2 } \left(\sum _ { i = 1 } ^ { N } \lambda _ { i }  t _ { i } \mathbf { x } _ { i }\right) \cdot \left(\sum _ { j = 1 } ^ { N } \lambda _ { j }  t _ { j } \mathbf { x } _ { j }\right) - \sum _ { i = 1 } ^ { N } \lambda _ { i } t _ { i } \left( \sum _ { j = 1 } ^ { N } \lambda _ { j }  t _ { j } \mathbf { x } _ { j }\right)  \cdot \mathbf { x } _ { i } - b\sum _ { i = 1 } ^ { N }  \lambda _ { i } t _ { i } + \sum _ { i = 1 } ^ { N } \lambda_i
$$

We see that the second last term is $0$ from our derivative with respect to $b$. Therfore, our simplified Lagrangian looks as follows:

$$
\widetilde { L } ( \mathbf { \lambda } ) = \sum _ { i = 1 } ^ { N } \lambda _ { i } - \frac { 1 } { 2 } \sum _ { i = 1 } ^ { N } \sum _ { j = 1 } ^ { N } \lambda _ { i } \lambda _ { j } t _ { i } t _ { j }\left( \mathbf { x } _ { i } \cdot \mathbf { x } _ { j } \right)
$$

which is known as the *dual representation* of the maximum margin problem. Note that this problem is convex and only relies on the dot product of the pairs of samples. 

The real advantage comes when we represent our original transformation as:

$$
y ( \mathbf { x } ) = \mathbf { w } ^ { \mathrm { T } } \phi ( \mathbf { x } ) + b
$$

where $\phi(\mathbf{x})$ is some feature space transformation. Using all the same math, we arrive at a similar dual representation:

$$
\widetilde { L } ( \mathbf { \lambda } ) = \sum _ { i = 1 } ^ { N } \lambda _ { i } - \frac { 1 } { 2 } \sum _ { i = 1 } ^ { N } \sum _ { j = 1 } ^ { N } \lambda _ { i } \lambda _ { j } t _ { i } t _ { j } k\left( \mathbf { x } _ { i } , \mathbf { x } _ { j } \right)
$$

where the Kernal function is defined such that $k\left( \mathbf { x } _ { i } , \mathbf { x } _ { j } \right) = \phi(\mathbf{x}_i) \cdot \phi(\mathbf{x}_j)$. Similarly, our original decision rule becomes (after plugging $\mathbf{w}$ back in),

$$
y ( \mathbf { x } ) = \sum _ { i = 1 } ^ { N } \lambda _ { n } t _ { i } k \left( \mathbf { x } , \mathbf { x } _ { i } \right) + b
$$

What this means is that we don't need to know the individual transformation of each vector in another feature-space. Instead, we just need to know how the dot product transforms. This is really useful as we will see later.